In [ ]:
import pandas as pd

from token import Token
from titlequery import TitleQuery

In [ ]:
# Create token and title query instances
token = Token()
title_query = TitleQuery(token)

In [ ]:
# Query any instrument
queried_data = title_query.get_instrument_data(instrument_type="cedears", country="argentina")

In [ ]:
# Get it into a df
queried_df = pd.DataFrame(queried_data['titulos'])

In [ ]:
# Expand the puntas column
queried_df[[key for key in queried_df['puntas'].iloc[0].keys()]] = queried_df['puntas'].apply(lambda x: pd.Series(x))
queried_df.head(2)

In [ ]:
queried_df = queried_df[['simbolo', 'ultimoPrecio', 'volumen', 'moneda', 'descripcion', 'plazo', 'precioVenta', 'precioCompra', 'cantidadCompra', 'cantidadVenta']].copy()
queried_df.head(2)

In [ ]:
# Create a df with the species in ARS
df_pesos = queried_df[queried_df['moneda'] == "1"].copy()

# Filter the species in dollars in dollar D (not cable)
letter = "D"
df_dolares = queried_df[(queried_df['moneda'] == "2") & (queried_df['simbolo'].str.endswith(letter))].copy()

In [ ]:
# Join the dfs in ARS ans USD

# Add the name of the species in dollars to be able to join back into them
 # at one point it would be good to have the actual names of the species in dollars, because maybe they are different as the species name in pesos with a D in the end
df_pesos.loc[:,f'simbolo_{letter}'] = df_pesos["simbolo"] + letter

# Join them
df_pesos_and_dollars = pd.merge(df_pesos, df_dolares, left_on='simbolo_D', right_on="simbolo" ,how='left', suffixes=('', '_D'))

In [ ]:
# Calculate the new columns MEP
df_pesos_and_dollars.loc[:, 'MEP'] = df_pesos_and_dollars['ultimoPrecio'] / df_pesos_and_dollars['ultimoPrecio_D']
df_pesos_and_dollars.loc[:, "MEP_venta_D"] = df_pesos_and_dollars["precioCompra"] / df_pesos_and_dollars["precioVenta_D"]
df_pesos_and_dollars.loc[:, "MEP_compra_P"] = df_pesos_and_dollars["precioVenta"] / df_pesos_and_dollars["precioCompra_D"]

In [ ]:
final_filter_volume = df_pesos_and_dollars[(df_pesos_and_dollars.volumen > 1000) & (df_pesos_and_dollars.volumen_D > 100)]
final_filter_volume.sort_values(by="MEP", ascending=False)

In [ ]:
cols = ["simbolo","descripcion","volumen","ultimoPrecio", "precioVenta","precioCompra","cantidadCompra","cantidadVenta","volumen_D", "ultimoPrecio_D", "precioVenta_D","precioCompra_D","cantidadCompra_D","cantidadVenta_D","MEP", "MEP_venta_D", "MEP_compra_P"]
final_filter_volume[cols].to_csv("data/outputs/cedear_data_iol.csv", index=False)